# Fraud Detection Model Building (Modularized)

This notebook demonstrates using the modular fraud detection framework with:
- Automated data loading and validation
- Feature engineering pipeline
- Class imbalance handling (SMOTE, class weighting)
- Comprehensive evaluation metrics (ROC AUC, PR AUC, confusion matrix)
- MLflow experiment tracking
- Model explainability (SHAP, LIME)

In [1]:
import sys
import os

sys.path.insert(0, os.path.abspath('..'))

from scripts.data.loader import DataLoader
from scripts.features.engineering import FeatureEngineer, prepare_fraud_data, prepare_credit_data
from scripts.models.train import ModelTrainer, ModelFactory, ResamplingStrategy
from scripts.models.evaluate import ModelEvaluator, compare_models
from scripts.models.explainability import ModelExplainer

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
sns.set(style="whitegrid")

print("✓ All modules imported successfully!")

✓ All modules imported successfully!


In [5]:

loader = DataLoader(data_dir='../data')
fraud_data, credit_data = loader.load_both_datasets(validate=True)

print(f"\n✓ Fraud dataset loaded: {fraud_data.shape}")
print(f"✓ Credit dataset loaded: {credit_data.shape}")

INFO:scripts.data.loader:DataLoader initialized with data directory: ../data
INFO:scripts.data.loader:Loading cleaned fraud data from ../data\cleaned_data_1.csv
INFO:scripts.data.loader:Loaded fraud dataset: 138846 rows, 18 columns
INFO:scripts.data.loader:
INFO:scripts.data.loader:Validating Fraud Data
INFO:scripts.data.loader:==================================================
INFO:scripts.data.loader:Fraud Data: No missing values found
INFO:scripts.data.loader:Fraud Data: No duplicates found
INFO:scripts.data.loader:Fraud Data: Data types - {dtype('int64'): 11, dtype('float64'): 4, dtype('O'): 3}
INFO:scripts.data.loader:Fraud Data: Class distribution:
INFO:scripts.data.loader:  - Class 0: 125849 (90.64%)
INFO:scripts.data.loader:  - Class 1: 12997 (9.36%)
INFO:scripts.data.loader:  - Imbalance ratio: 9.68:1
INFO:scripts.data.loader:==================================================

INFO:scripts.data.loader:Loading cleaned credit data from ../data\cleaned_data_2.csv
INFO:scripts.dat


✓ Fraud dataset loaded: (138846, 18)
✓ Credit dataset loaded: (283726, 31)


In [7]:

X_fraud, y_fraud = prepare_fraud_data(fraud_data, target_col='class')

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_fraud, y_fraud, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_fraud
)

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")
print(f"\nClass distribution (training):")
print(y_train.value_counts())
print(f"\nClass imbalance ratio: {y_train.value_counts()[0] / y_train.value_counts()[1]:.2f}:1")

Training set: (111076, 17)
Test set: (27770, 17)

Class distribution (training):
class
0    100678
1     10398
Name: count, dtype: int64

Class imbalance ratio: 9.68:1


In [ ]:

engineer = FeatureEngineer()
X_train_prep, X_test_prep = engineer.prepare_features(X_train, X_test, scale=True, encode=True)

print(f"✓ Feature engineering complete!")
print(f"Prepared training set: {X_train_prep.shape}")
print(f"Prepared test set: {X_test_prep.shape}")
print(f"Feature names: {engineer.feature_names[:10]}...") 

INFO:scripts.features.engineering:Starting feature preparation pipeline
INFO:scripts.features.engineering:Encoding 3 categorical columns: ['signup_time', 'purchase_time', 'device_id']
INFO:scripts.features.engineering:  - signup_time: Label encoding (111076 unique values)


In [ ]:
# Initialize trainer with MLflow
trainer = ModelTrainer(experiment_name="fraud_detection_notebook")

# Define model configurations to test
model_configs = {
    'rf_balanced': {
        'model_name': 'random_forest',
        'resampling': 'none',
        'use_class_weight': True,
        'params': {'n_estimators': 100, 'max_depth': 10, 'random_state': 42}
    },
    'rf_smote': {
        'model_name': 'random_forest',
        'resampling': 'smote',
        'use_class_weight': False,
        'params': {'n_estimators': 100, 'max_depth': 10, 'random_state': 42}
    },
    'gb_balanced': {
        'model_name': 'gradient_boosting',
        'resampling': 'none',
        'use_class_weight': False,  # GB doesn't support class_weight
        'params': {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 5, 'random_state': 42}
    },
    'lr_smote_tomek': {
        'model_name': 'logistic_regression',
        'resampling': 'smote_tomek',
        'use_class_weight': False,
        'params': {'C': 1.0, 'random_state': 42}
    }
}

print("Training models with different configurations...")
print("This may take a few minutes...\n")

In [ ]:
# Extract metrics for comparison
metrics_dict = {name: metrics for name, (_, metrics) in results.items()}

# Compare models
comparison_df = compare_models(metrics_dict)
print("\nModel Comparison:")
print(comparison_df)

# Visualize comparison
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

metrics_to_plot = ['precision', 'recall', 'f1_score', 'roc_auc']
titles = ['Precision', 'Recall', 'F1 Score', 'ROC AUC']

for idx, (metric, title) in enumerate(zip(metrics_to_plot, titles)):
    ax = axes[idx // 2, idx % 2]
    comparison_df[metric].plot(kind='barh', ax=ax, color='skyblue')
    ax.set_title(f'{title} Comparison')
    ax.set_xlabel(title)
    ax.set_xlim([0, 1])
    ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

# Find best model
best_model_name = comparison_df['f1_score'].idxmax()
print(f"\n✓ Best model by F1-Score: {best_model_name}")
print(f"  F1: {comparison_df.loc[best_model_name, 'f1_score']:.4f}")
print(f"  Recall: {comparison_df.loc[best_model_name, 'recall']:.4f}")
print(f"  Precision: {comparison_df.loc[best_model_name, 'precision']:.4f}")

In [ ]:
# LIME: Explain individual fraud predictions
print("Generating LIME explanations for sample fraud cases...")

# Find a few fraud cases
fraud_indices = y_test[y_test == 1].index[:3]

for i, idx in enumerate(fraud_indices):
    instance = X_test_prep.loc[idx].values
    print(f"\nExplaining fraud case {i+1}:")
    explanation = explainer.explain_with_lime(instance, instance_index=i, num_features=10)

In [ ]:
# Initialize explainer
explainer = ModelExplainer(best_model, X_train_prep)

# SHAP Summary Plot (global feature importance)
print("Generating SHAP summary plot...")
shap_values = explainer.explain_with_shap(
    X_test_prep.sample(min(100, len(X_test_prep))),
    plot_type='summary',
    max_display=15
)

# Feature importance comparison
print("\nFeature Importance:")
importance_df = explainer.get_feature_importance(method='shap', top_n=15)
explainer.plot_feature_importance(method='shap', top_n=15, figsize=(10, 8))

## 6. Model Explainability (SHAP & LIME)

Understanding **why** the model makes predictions:

In [ ]:
# Find optimal threshold for business requirements
print("Threshold Analysis:")
print("="*60)

# F1-optimized threshold
optimal_f1 = evaluator.find_optimal_threshold(y_test, y_pred_proba, metric='f1')

# Precision-targeted threshold (e.g., 85% precision requirement)
precision_target = evaluator.evaluate_at_precision_target(y_test, y_pred_proba, target_precision=0.85)

print("\nSummary:")
print(f"Default threshold (0.5): Recall = {best_metrics['recall']:.4f}, Precision = {best_metrics['precision']:.4f}")
print(f"F1-optimized threshold ({optimal_f1['threshold']:.3f}): Recall = {optimal_f1['recall']:.4f}, Precision = {optimal_f1['precision']:.4f}")
print(f"Precision-targeted ({precision_target['threshold']:.3f}): Recall = {precision_target['recall']:.4f}, Precision = {precision_target['precision']:.4f}")

In [ ]:
# Get best model
best_model, best_metrics = results[best_model_name]

# Make predictions
y_pred = best_model.predict(X_test_prep)
y_pred_proba = best_model.predict_proba(X_test_prep)[:, 1]

# Create evaluator
evaluator = ModelEvaluator(best_model_name)

# Plot confusion matrix
fig = evaluator.plot_confusion_matrix(y_test, y_pred, figsize=(8, 6))
plt.show()

# Plot ROC curve
fig = evaluator.plot_roc_curve(y_test, y_pred_proba, figsize=(8, 6))
plt.show()

# Plot PR curve
fig = evaluator.plot_precision_recall_curve(y_test, y_pred_proba, figsize=(8, 6))
plt.show()

## 5. Detailed Evaluation of Best Model

Generate comprehensive evaluation reports including:
- Confusion Matrix
- ROC Curve
- Precision-Recall Curve
- Optimal Threshold Analysis

## 4. Compare Model Performance

Comprehensive metrics comparison including:
- **Accuracy, Precision, Recall, F1-Score**
- **ROC AUC**: Area under ROC curve (overall discriminative power)
- **PR AUC**: Area under Precision-Recall curve (critical for imbalanced data)

In [ ]:
# Train all models
results = trainer.train_multiple_models(
    model_configs=model_configs,
    X_train=X_train_prep,
    y_train=y_train,
    X_test=X_test_prep,
    y_test=y_test,
    output_dir='../models'
)

print("\n✓ All models trained successfully!")

## 3. Train Models with Class Imbalance Handling

Training multiple models with different configurations:
- **Class weighting**: `class_weight='balanced'`
- **SMOTE**: Synthetic Minority Over-sampling Technique
- **SMOTETomek**: SMOTE + Tomek links for better boundaries

## 2. Prepare Features

Using FeatureEngineer for automated preprocessing, encoding, and scaling.

## 1. Load and Validate Data

Using the modular DataLoader to load datasets with built-in validation.

## Data Preparation

In [13]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
import mlflow
import mlflow.sklearn
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

# Set plot style for better visuals
sns.set(style="whitegrid")


In [14]:
# Load the datasets
fraud_data = pd.read_csv('../data/cleaned_data_1.csv')
credit_data = pd.read_csv('../data/cleaned_data_2.csv')

In [ ]:
# Prepare data for the model (e-commerce)
X1 = fraud_data.drop(columns=['class'])  
X2 = credit_data.drop(columns=['Class'])  

y1 = fraud_data['class']  
y2 = credit_data['Class']  


print(np.unique(y1, return_counts=True))
print(np.unique(y2, return_counts=True))

(array([0, 1]), array([125849,  12997]))
(array([0, 1]), array([283253,    473]))


In [17]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)

X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=42)


In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Check for non-numeric columns in X_train1
non_numeric_columns = X_train1.select_dtypes(include=['object']).columns
print("Non-numeric columns:", non_numeric_columns)

# Convert categorical columns to numeric (if any)
for column in non_numeric_columns:
    # If it's a binary categorical column (like 'sex'), use LabelEncoder
    if len(X_train1[column].unique()) <= 2:
        label_encoder = LabelEncoder()
        X_train1[column] = label_encoder.fit_transform(X_train1[column])
        X_test1[column] = label_encoder.transform(X_test1[column])
    else:
        # For columns with more than two categories, use OneHotEncoder
        X_train1 = pd.get_dummies(X_train1, columns=[column], drop_first=True)
        X_test1 = pd.get_dummies(X_test1, columns=[column], drop_first=True)

# Now fit the Logistic Regression model
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train1, y_train1)
y_pred_log_reg = log_reg.predict(X_test1)

# Print classification report
print("Logistic Regression (fraud data):")
print(classification_report(y_test1, y_pred_log_reg))


In [27]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train1, y_train1)
y_pred_log_reg = log_reg.predict(X_test1)
print("Logistic Regression (fraud data):")
print(classification_report(y_test1, y_pred_log_reg))

Logistic Regression (fraud data):
              precision    recall  f1-score   support

           0       0.91      1.00      0.95     25193
           1       0.00      0.00      0.00      2577

    accuracy                           0.91     27770
   macro avg       0.45      0.50      0.48     27770
weighted avg       0.82      0.91      0.86     27770



In [30]:
# Decision Tree Classifier
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train1, y_train1)
y_pred_dt = dt_clf.predict(X_test1)
print("Decision Tree (fraud data):")
print(classification_report(y_test1, y_pred_dt))

Decision Tree (fraud data):
              precision    recall  f1-score   support

           0       0.96      0.94      0.95     25193
           1       0.50      0.58      0.54      2577

    accuracy                           0.91     27770
   macro avg       0.73      0.76      0.74     27770
weighted avg       0.91      0.91      0.91     27770



In [31]:
# Random Forest Classifier (already implemented)
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train1, y_train1)
y_pred_rf = rf_clf.predict(X_test1)
print("Random Forest (fraud data):")
print(classification_report(y_test1, y_pred_rf))

Random Forest (fraud data):
              precision    recall  f1-score   support

           0       0.96      1.00      0.98     25193
           1       1.00      0.55      0.71      2577

    accuracy                           0.96     27770
   macro avg       0.98      0.77      0.84     27770
weighted avg       0.96      0.96      0.95     27770



In [32]:
# Gradient Boosting Classifier
gb_clf = GradientBoostingClassifier(n_estimators=50, learning_rate=0.1, max_depth=3, random_state=42)
gb_clf.fit(X_train1, y_train1)
y_pred_gb = gb_clf.predict(X_test1)
print("Gradient Boosting (fraud data):")
print(classification_report(y_test1, y_pred_gb))

Gradient Boosting (fraud data):
              precision    recall  f1-score   support

           0       0.96      1.00      0.98     25193
           1       1.00      0.55      0.71      2577

    accuracy                           0.96     27770
   macro avg       0.98      0.77      0.84     27770
weighted avg       0.96      0.96      0.95     27770



In [34]:
# Multi-Layer Perceptron (MLP)
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)
mlp_clf.fit(X_train1, y_train1)
y_pred_mlp = mlp_clf.predict(X_test1)
print("MLP Classifier (fraud data):")
print(classification_report(y_test1, y_pred_mlp))

MLP Classifier (fraud data):
              precision    recall  f1-score   support

           0       0.91      1.00      0.95     25193
           1       0.10      0.00      0.00      2577

    accuracy                           0.91     27770
   macro avg       0.50      0.50      0.48     27770
weighted avg       0.83      0.91      0.86     27770



In [ ]:
# RNN Model
from keras.layers import SimpleRNN
rnn_model = Sequential()
rnn_model.add(SimpleRNN(64, activation='relu', input_shape=(X_train1.shape[1], 1)))
rnn_model.add(Dense(1, activation='sigmoid'))
rnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
rnn_model.fit(X_train1.values.reshape(-1, X_train1.shape[1], 1), y_train1, epochs=10, batch_size=32)
y_pred_rnn = (rnn_model.predict(X_test1.values.reshape(-1, X_test1.shape[1], 1)) > 0.5).astype(int)
print("RNN (fraud data):")
print(classification_report(y_test1, y_pred_rnn))

..............................................

## Experiments

In [33]:
datasets = {
    "fraud_data": (X_train1, y_train1, X_test1, y_test1),
    "credit_data": (X_train2, y_train2, X_test2, y_test2),
}

In [2]:
# Defining each model
models = [
    # Logistic Regression
    (
        "Logistic Regression", 
        LogisticRegression(C=1, solver='liblinear')
    ),
    
    # Random Forest
    (
        "Random Forest", 
        RandomForestClassifier(n_estimators=30, max_depth=3)
    ),

    # Decision Tree
    (
        "Decision Tree", 
        DecisionTreeClassifier(max_depth=5)
    ),
    
    # Multi-Layer Perceptron (MLP)
    (
        "MLP Classifier", 
        MLPClassifier(hidden_layer_sizes=(100,), max_iter=300)
    ),
    
    # # Convolutional Neural Network (CNN)
    # (
    #     "CNN", 
    #     Sequential([
    #         Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    #         Flatten(),
    #         Dense(128, activation='relu'),
    #         Dense(1 , activation='sigmoid')
    #     ])
    # ),
    
    # # Recurrent Neural Network (RNN)
    # (
    #     "RNN", 
    #     Sequential([
    #         SimpleRNN(50, input_shape=(100, 1), activation='relu'),
    #         Dense(1, activation='sigmoid')
    #     ])
    # ),
    
    # # Long Short-Term Memory (LSTM)
    # (
    #     "LSTM", 
    #     Sequential([
    #         LSTM(50, input_shape=(100, 1), activation='relu'),
    #         Dense(1, activation='sigmoid')
    #     ])
    # )
]


NameError: name 'LogisticRegression' is not defined